In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from LABNET import Teacher

First hurdle. the embedding layer.  It requires the ints as inputs, not one hot.  i need one hot.  i will also need to rewire the LLM i use to do the same.  hmm.  save the weights, load it into a model that has the same shapes, but accepts one hot arrays, not vocab_indexes.

In [3]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, hidden_dim, num_layers, dropout):
        super(TransformerEncoder, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim, dropout),
            num_layers
        )

    def forward(self, x):
        return self.transformer(x)

class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout):
        super(SimpleLanguageModel, self).__init__()

        # Define the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Define the transformer encoder
        self.transformer_encoder = TransformerEncoder(embedding_dim, num_heads, hidden_dim, num_layers, dropout)

        # Define the output layer
        self.output_layer = nn.Linear(embedding_dim, class_num)

    def forward(self, input_data):
        # Input_data is of shape (batch_size, sequence_length)
        # Apply embedding layer
        #print(input_data.shape)
        embedded = self.embedding(input_data)
        #print(embedded.shape)
        # Pass through the transformer encoder
        transformed = self.transformer_encoder(embedded)

        # Average along the sequence dimension
        avg_transformed = torch.mean(transformed, dim=1)

        # Apply the output layer
        output = self.output_layer(avg_transformed)

        return output
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
embedding_dim = 24
num_heads = 8
hidden_dim  = 20
num_layers = 2
dropout = 0.1
vocab_size = 2_000
class_num = 100
batch_size = 23
sequence_length = 100

In [5]:
SLM = SimpleLanguageModel(vocab_size, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout)

In [6]:
input_data = random_data = torch.randint(low=0, high=vocab_size, size=(batch_size, sequence_length))
SLM(input_data).shape

torch.Size([23, 100])

In [7]:
count_parameters(SLM)

57500

In [8]:
teacher_slm = Teacher(SLM,(sequence_length,)) #don't specify batch!!

In [ ]:
teacher_slm.configure(dist_type = "ints", gen_m = vocab_size) #this is dying.  might be time for colab!!

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


In [ ]:
import numpy as np
torch.from_numpy(np.random.randint(0, high=256, size=(10,100)))